In [13]:
%run metrics.py

In [14]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

def plot_voxels_side_by_side(voxel_datasets, titles, rows=0, cols=0, box_size=50):
    """
    Plot multiple 3D voxel grids in a single figure with a compact layout.

    Parameters:
    voxel_datasets (list of np.ndarray): List of 3D arrays representing voxel grids.
    titles (list of str): List of titles for the subplots.
    """
    if cols == 0 or rows == 0:
        num_plots = len(voxel_datasets)
        rows = int(np.ceil(num_plots / 3))  # 3-column layout
        cols = 3  

    fig = make_subplots(rows=rows, cols=cols, 
                        specs=[[{'type': 'scatter3d'}] * cols] * rows,
                        subplot_titles=titles)


    for i, (voxel_data, title) in enumerate(zip(voxel_datasets, titles)):
        row = (i // cols) + 1
        col = (i % cols) + 1
        x, y, z = np.where(voxel_data > 0)
        values = voxel_data[x, y, z] 

        scatter = go.Scatter3d(
            x=x, y=y, z=z,
            mode='markers',
            marker=dict(size=3, symbol='square', color=values, colorscale='Viridis', showscale=False),
            name=title
        )
        fig.add_trace(scatter, row=row, col=col)
        
        fig.update_scenes(
            dict(
                xaxis=dict(range=[0, box_size], title="X"),
                yaxis=dict(range=[0, box_size], title="Y"),
                zaxis=dict(range=[0, box_size], title="Z"),
                aspectratio=dict(x=1, y=1, z=1),
                aspectmode='cube'
            ),
            row=row, col=col
        )

    #Adjust height and width of the figure
    fig.update_layout(
        height=300 * rows,   
        width=400 * cols,    
        title="Voxel Grids",
        margin=dict(l=10, r=10, t=50, b=10)  # Reduce padding around the plots
    )
    
    # config = {"displayModeBar": False}
    
    fig.show()


In [15]:
def compare_deferred(metrics):    #borrowed from the Dark side
    names = []
    elements = []
    symmetry_checks = {}

    def append(name, ab):
        names.append(name)
        elements.append(ab)
        for metric_name, metric_fn in metrics:      #symmetry check
            value_ab = metric_fn(ab[0], ab[1])
            value_ba = metric_fn(ab[1], ab[0])
            # print(metric_name, value_ab, value_ba)
            symmetry_checks[(name, metric_name)] = 1 if value_ab == value_ba else 0
        return [metric(*ab) for name, metric in metrics]

    def table():
        # Determine column widths for formatting
        longest_metric_name_len = max(len(name) for name, _ in metrics)
        longest_name_len = max(max(len(name) for name in names), 6)

        print(" " * longest_metric_name_len, "|", end=" ")
        for name in names:
            print(name + " " * (longest_name_len - len(name)), "|", end=" ")
        print("Symmetry" + " " * (longest_name_len - 8), "|")

        print("-" * (longest_metric_name_len + 2 + (len(names) + 1) * (longest_name_len + 3)))

        for metric_name, metric_fn in metrics:
            print(" " * (longest_metric_name_len - len(metric_name)) + metric_name, "|", end=" ")
            for i, (a, b) in enumerate(elements):
                value = repr(round(metric_fn(a, b), min(longest_name_len - 2, 4)))
                print(value + " " * (longest_name_len - len(value)), "|", end=" ")
            symmetry = all(symmetry_checks[(name, metric_name)] for name in names)
            print(f"{symmetry}       |")

    return append, table


metrics = [
    ("TVD", total_variation_distance),
    ("Wasserstein 3D optimized", wasserstein_distance_3d_optimized),
    ("Wasserstein 3D optimized and normalized", wasserstein_distance_3d_optimized_normalized),
    ("Q-Score", compute_q_score)
]

# left, right porownanie

In [16]:
blob1 = np.load("v_blob1.npy")
blob_right = np.load("v_blob_right.npy")
blob_left = np.load("v_blob_left.npy")

voxel_pairs = [
    ("normal", blob1),
    ("right", blob_right),
    ("left", blob_left),
]
compare_append, compare_table = compare_deferred(metrics)

#All comparisons
for (name, v1) in voxel_pairs:
    compare_append(f"Basic to {name}", [blob1, v1])

compare_append(f"right to basic", [blob_right, blob1]) # code says that its not symmetric, yet data says that it is....
compare_append(f"left to basic", [blob_left, blob1])
compare_table()


voxel_names, voxel_data = zip(*voxel_pairs)
plot_voxels_side_by_side(voxel_data, voxel_names, 2, 4)


                                        | Basic to normal | Basic to right  | Basic to left   | right to basic  | left to basic   | Symmetry        |
-----------------------------------------------------------------------------------------------------------------------------------------------------
                                    TVD | 1.0             | 0.9961          | 0.9962          | 0.9961          | 0.9962          | True       |
               Wasserstein 3D optimized | 1.0             | 0.9796          | 0.9871          | 0.9796          | 0.9871          | False       |
Wasserstein 3D optimized and normalized | 1.0             | 0.9214          | 0.9391          | 0.9214          | 0.9391          | False       |
                                Q-Score | 1.0             | 0.9286          | 0.9308          | 0.9286          | 0.9308          | True       |


In [17]:
# big1 = np.load("v_big1.npy")
# big_right = np.load("v_big_right.npy")
# big_left = np.load("v_big_left.npy")



# voxel_pairs = [
#     ("big right", big_right),
#     ("big left", big_left),
# ]
# compare_append, compare_table = compare_deferred(metrics)

# #All comparisons
# for (name, v1) in voxel_pairs:
#     compare_append(f"Basic to {name}", [big1, v1])

# compare_table()


# voxel_names, voxel_data = zip(*voxel_pairs)
# plot_voxels_side_by_side(voxel_data, voxel_names, 2, 4)


In [18]:
clean = np.load("v_clean.npy")
background = np.load("v_background.npy")
cluttered = np.load("v_cluttered.npy")

voxel_pairs = [
    ("clean", clean),
    ("background", background),
    ('cluttered', cluttered)
]
compare_append, compare_table = compare_deferred(metrics)

compare_append(f"Clean to background", [clean, background])
compare_append(f"Clean to cluttered", [clean, cluttered])

compare_table()


voxel_names, voxel_data = zip(*voxel_pairs)
plot_voxels_side_by_side(voxel_data, voxel_names, 2, 4)


                                        | Clean to background | Clean to cluttered  | Symmetry            |
-----------------------------------------------------------------------------------------------------------
                                    TVD | 0.9946              | 0.9523              | True       |
               Wasserstein 3D optimized | 0.9786              | 0.9129              | False       |
Wasserstein 3D optimized and normalized | 0.9531              | 0.8184              | False       |
                                Q-Score | 0.9293              | 0.5632              | True       |
